In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib
matplotlib.use('TkAgg')  # Pour afficher dans la cellule VS Code ou Notebook
import matplotlib.pyplot as plt

In [2]:
# 🔹 1. Import dataset and create smoothed throughput using EMA (Exponential Moving Average)
file_path = 'prepared_5G_data.csv'
df = pd.read_csv(file_path)
df['Smooth_Throughput'] = df['Throughput'].ewm(span=5, adjust=False).mean()

In [3]:
# 🔹 2. Préparation des features (X) et target (y)
features = df.drop(columns=['Throughput', 'Smooth_Throughput'])
target = df['Smooth_Throughput']

In [4]:
# Split des données
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [5]:
# 🔹 3. Modèle CNN + LSTM (1D expansion)
model = Sequential([
    Reshape((features.shape[1], 1), input_shape=(features.shape[1],)),
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    LSTM(64, activation='tanh'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

C:\Users\axxiv\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
# 🔹 4. Entraînement
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)
    ],
    verbose=1
)

Epoch 1/50
682/682 ━━━━━━━━━━━━━━━━━━━━ 27s 29ms/step - loss: 332102.2812 - mean_absolute_error: 427.4677 - root_mean_squared_error: 573.0073 - val_loss: 136831.5938 - val_mean_absolute_error: 281.8919 - val_root_mean_squared_error: 369.9076
Epoch 2/50
682/682 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 134988.6406 - mean_absolute_error: 281.9771 - root_mean_squared_error: 367.3699 - val_loss: 106190.5078 - val_mean_absolute_error: 239.8963 - val_root_mean_squared_error: 325.8688
Epoch 3/50
682/682 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - loss: 103937.2188 - mean_absolute_error: 243.1910 - root_mean_squared_error: 322.3487 - val_loss: 80360.9609 - val_mean_absolute_error: 210.5976 - val_root_mean_squared_error: 283.4801
Epoch 4/50
682/682 ━━━━━━━━━━━━━━━━━━━━ 25s 37ms/step - loss: 78250.8594 - mean_absolute_error: 206.6665 - root_mean_squared_error: 279.6599 - val_loss: 66555.8359 - val_mean_absolute_error: 184.8138 - val_root_mean_squared_error: 257.9842
Epoch 5/50
682/682 ━━━━━━━━━━━━━━━━

In [7]:
# 🔹 5. Évaluation
loss, rmse, mae = model.evaluate(X_test, y_test)
print(f"Test RMSE: {rmse:.2f} | MAE: {mae:.2f}")

426/426 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 47369.1367 - mean_absolute_error: 154.0721 - root_mean_squared_error: 217.6037
Test RMSE: 214.79 | MAE: 151.87


In [8]:
# 🔹 6. Visualisation des prédictions réelles vs prédites
plt.figure(figsize=(10, 4))
y_pred_all = model.predict(X_test).flatten()
plt.plot(y_test.values[:100], label='Réel', color='blue')
plt.plot(y_pred_all[:100], label='Prédit', color='orange')
plt.title("Prédictions vs Réel (100 premiers exemples)")
plt.xlabel("Index")
plt.ylabel("Débit lissé (EMA)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

426/426 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step


In [ ]:
# 🔹 7. Courbes d'apprentissage
plt.figure(figsize=(10, 5))
plt.plot(history.history['val_mean_absolute_error'], label='Validation MAE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.title("Apprentissage du modèle CNN + LSTM sur débit lissé (EMA)")
plt.xlabel("Épochs")
plt.ylabel("Erreur")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()